### Association rules Breakfast
#### Department - Breakfast

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [2]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [3]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [4]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [5]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [6]:
combine_train_prior

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods
2,2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,fresh vegetables,produce
3,3,1,22035,8,1,112108,train,4,4,10,9.0,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
4,4,1,47209,7,0,112108,train,4,4,10,9.0,Organic Hass Avocado,24,4,fresh fruits,produce
5,5,1,13176,6,0,112108,train,4,4,10,9.0,Bag of Organic Bananas,24,4,fresh fruits,produce
6,6,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
7,7,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,fresh vegetables,produce
8,8,36,46979,8,1,79431,train,23,6,18,30.0,Asparagus,83,4,fresh vegetables,produce
9,9,36,39612,1,0,79431,train,23,6,18,30.0,Grated Pecorino Romano Cheese,2,16,specialty cheeses,dairy eggs


In [7]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="breakfast"].reset_index()

In [8]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [9]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=4)

In [10]:
grouped_data

,order_id,product_name
83,10526,Organic Cinnamon Crunch Cereal
84,10526,Healthy Grains Oats & Honey Clusters with Toas...
85,10526,Raisin Bran Cereal
86,10526,Vanilla Blueberry Clusters With Flax Seeds Gra...
87,10526,Organic Honey Nut O's Cereal
124,14043,Honey Nut Cheerios
125,14043,Raisin Bran Cereal
126,14043,Organic Cinnamon Crunch Cereal
127,14043,Organic Razzi Raspberry Frosted Toaster Pastri...
204,24090,Maple & Brown Sugar Instant Oatmeal


In [11]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [12]:
#relace NAN with 0
table1=table1.fillna(0)

In [13]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1


In [14]:
basket_sets = table1.applymap(encode_units)

In [15]:
## Running Apriori to get frequent itemset with a minimum support of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.01, use_colnames=True)

In [16]:
frequent_itemsets

,support,itemsets
0,0.013150,[90 Calorie Chocolate Fudge Brownies]
1,0.020688,[Ancient Grain Blueberry Hemp Granola]
2,0.011643,[Ancient Grain Granola Cranberry Pecan]
3,0.026552,[Ancient Grain Original Granola]
4,0.015328,[Apple Cinnamon Cheerios Cereal]
5,0.015998,[Apple Cinnamon Instant Oatmeal]
6,0.011810,[Apple Jacks Cereal]
7,0.013988,[Apples & Cinnamon Instant Oatmeal]
8,0.018259,[BelVita Blueberry Breakfast Biscuit Packs]
9,0.012480,[BelVita Golden Oat Breakfast Biscuit Packets]


In [17]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [18]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Nutri-Grain Soft Baked Strawberry Cereal Brea...,(Kellogg's Nutri-Grain Apple Cinnamon Cereal),0.028897,0.020856,0.010302,0.356522,17.094430,0.009700,1.521643
1,(Kellogg's Nutri-Grain Apple Cinnamon Cereal),(Nutri-Grain Soft Baked Strawberry Cereal Brea...,0.020856,0.028897,0.010302,0.493976,17.094430,0.009700,1.919085
2,(Healthy Grains Oats & Honey Clusters with Toa...,(Healthy Grains Fiber Cinnamon Oat Clusters),0.015495,0.036016,0.010470,0.675676,18.760214,0.009912,2.972283
3,"(Healthy Grains Fiber Cinnamon Oat Clusters, V...",(Healthy Grains Oats & Honey Clusters with Toa...,0.019600,0.030153,0.010470,0.534188,17.715753,0.009879,2.082056
4,(Healthy Grains Oats & Honey Clusters with Toa...,(Vanilla Blueberry Clusters With Flax Seeds Gr...,0.012899,0.046821,0.010470,0.811688,17.335862,0.009866,5.061707
5,(Vanilla Blueberry Clusters With Flax Seeds Gr...,(Healthy Grains Oats & Honey Clusters with Toa...,0.046821,0.012899,0.010470,0.223614,17.335862,0.009866,1.271404
6,(Healthy Grains Oats & Honey Clusters with Toa...,"(Healthy Grains Fiber Cinnamon Oat Clusters, V...",0.030153,0.019600,0.010470,0.347222,17.715753,0.009879,1.501890
7,(Healthy Grains Fiber Cinnamon Oat Clusters),(Healthy Grains Oats & Honey Clusters with Toa...,0.036016,0.015495,0.010470,0.290698,18.760214,0.009912,1.387990
8,(Cinnamon Toast Crunch),(Cherrios Honey Nut),0.028729,0.037775,0.017673,0.615160,16.284699,0.016588,2.500326
9,(Cherrios Honey Nut),(Cinnamon Toast Crunch),0.037775,0.028729,0.017673,0.467849,16.284699,0.016588,1.825179


In [19]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Nutri-Grain Soft Baked Strawberry Cereal Brea...,(Kellogg's Nutri-Grain Apple Cinnamon Cereal),0.028897,0.020856,0.010302,0.356522,17.094430,0.009700,1.521643
1,(Kellogg's Nutri-Grain Apple Cinnamon Cereal),(Nutri-Grain Soft Baked Strawberry Cereal Brea...,0.020856,0.028897,0.010302,0.493976,17.094430,0.009700,1.919085
2,(Healthy Grains Oats & Honey Clusters with Toa...,(Healthy Grains Fiber Cinnamon Oat Clusters),0.015495,0.036016,0.010470,0.675676,18.760214,0.009912,2.972283
3,"(Healthy Grains Fiber Cinnamon Oat Clusters, V...",(Healthy Grains Oats & Honey Clusters with Toa...,0.019600,0.030153,0.010470,0.534188,17.715753,0.009879,2.082056
4,(Healthy Grains Oats & Honey Clusters with Toa...,(Vanilla Blueberry Clusters With Flax Seeds Gr...,0.012899,0.046821,0.010470,0.811688,17.335862,0.009866,5.061707
5,(Vanilla Blueberry Clusters With Flax Seeds Gr...,(Healthy Grains Oats & Honey Clusters with Toa...,0.046821,0.012899,0.010470,0.223614,17.335862,0.009866,1.271404
6,(Healthy Grains Oats & Honey Clusters with Toa...,"(Healthy Grains Fiber Cinnamon Oat Clusters, V...",0.030153,0.019600,0.010470,0.347222,17.715753,0.009879,1.501890
7,(Healthy Grains Fiber Cinnamon Oat Clusters),(Healthy Grains Oats & Honey Clusters with Toa...,0.036016,0.015495,0.010470,0.290698,18.760214,0.009912,1.387990
8,(Cinnamon Toast Crunch),(Cherrios Honey Nut),0.028729,0.037775,0.017673,0.615160,16.284699,0.016588,2.500326
9,(Cherrios Honey Nut),(Cinnamon Toast Crunch),0.037775,0.028729,0.017673,0.467849,16.284699,0.016588,1.825179


In [20]:
final_result.to_csv("Department_BreakfastApriori.csv",sep=',')

In [21]:
test1=pd.read_csv("Department_BreakfastApriori.csv")

In [22]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [23]:
test1=test1.replace({"u'": ''}, regex=True)

In [24]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [25]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [26]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [27]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Nutri-Grain Soft Baked Strawberry Cereal Break...,"u""Kelloggs Nutri-Grain Apple Cinnamon Cereal""",0.028897,0.020856,0.010302,0.356522,17.094430,0.009700,1.521643
1,"u""Kelloggs Nutri-Grain Apple Cinnamon Cereal""",Nutri-Grain Soft Baked Strawberry Cereal Break...,0.020856,0.028897,0.010302,0.493976,17.094430,0.009700,1.919085
2,Healthy Grains Oats & Honey Clusters with Toas...,Healthy Grains Fiber Cinnamon Oat Clusters,0.015495,0.036016,0.010470,0.675676,18.760214,0.009912,2.972283
3,"Healthy Grains Fiber Cinnamon Oat Clusters, Va...",Healthy Grains Oats & Honey Clusters with Toas...,0.019600,0.030153,0.010470,0.534188,17.715753,0.009879,2.082056
4,Healthy Grains Oats & Honey Clusters with Toas...,Vanilla Blueberry Clusters With Flax Seeds Gra...,0.012899,0.046821,0.010470,0.811688,17.335862,0.009866,5.061707
5,Vanilla Blueberry Clusters With Flax Seeds Gra...,Healthy Grains Oats & Honey Clusters with Toas...,0.046821,0.012899,0.010470,0.223614,17.335862,0.009866,1.271404
6,Healthy Grains Oats & Honey Clusters with Toas...,"Healthy Grains Fiber Cinnamon Oat Clusters, Va...",0.030153,0.019600,0.010470,0.347222,17.715753,0.009879,1.501890
7,Healthy Grains Fiber Cinnamon Oat Clusters,Healthy Grains Oats & Honey Clusters with Toas...,0.036016,0.015495,0.010470,0.290698,18.760214,0.009912,1.387990
8,Cinnamon Toast Crunch,Cherrios Honey Nut,0.028729,0.037775,0.017673,0.615160,16.284699,0.016588,2.500326
9,Cherrios Honey Nut,Cinnamon Toast Crunch,0.037775,0.028729,0.017673,0.467849,16.284699,0.016588,1.825179


In [28]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_BreakfastApriori.csv")

In [29]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Nutri-Grain Soft Baked Strawberry Cereal Break...,"u""Kelloggs Nutri-Grain Apple Cinnamon Cereal""",0.028897,0.020856,0.010302,0.356522,17.094430,0.009700,1.521643
1,"u""Kelloggs Nutri-Grain Apple Cinnamon Cereal""",Nutri-Grain Soft Baked Strawberry Cereal Break...,0.020856,0.028897,0.010302,0.493976,17.094430,0.009700,1.919085
2,Healthy Grains Oats & Honey Clusters with Toas...,Healthy Grains Fiber Cinnamon Oat Clusters,0.015495,0.036016,0.010470,0.675676,18.760214,0.009912,2.972283
3,"Healthy Grains Fiber Cinnamon Oat Clusters, Va...",Healthy Grains Oats & Honey Clusters with Toas...,0.019600,0.030153,0.010470,0.534188,17.715753,0.009879,2.082056
4,Healthy Grains Oats & Honey Clusters with Toas...,Vanilla Blueberry Clusters With Flax Seeds Gra...,0.012899,0.046821,0.010470,0.811688,17.335862,0.009866,5.061707
5,Vanilla Blueberry Clusters With Flax Seeds Gra...,Healthy Grains Oats & Honey Clusters with Toas...,0.046821,0.012899,0.010470,0.223614,17.335862,0.009866,1.271404
6,Healthy Grains Oats & Honey Clusters with Toas...,"Healthy Grains Fiber Cinnamon Oat Clusters, Va...",0.030153,0.019600,0.010470,0.347222,17.715753,0.009879,1.501890
7,Healthy Grains Fiber Cinnamon Oat Clusters,Healthy Grains Oats & Honey Clusters with Toas...,0.036016,0.015495,0.010470,0.290698,18.760214,0.009912,1.387990
8,Cinnamon Toast Crunch,Cherrios Honey Nut,0.028729,0.037775,0.017673,0.615160,16.284699,0.016588,2.500326
9,Cherrios Honey Nut,Cinnamon Toast Crunch,0.037775,0.028729,0.017673,0.467849,16.284699,0.016588,1.825179


In [30]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()